In [1]:
import numpy as np
import pandas as pd

#!pip install pyEDFlib
import pyedflib
#!pip install ipympl

from scipy.fftpack import fft, ifft, fftfreq
from scipy import signal as sg
from scipy.ndimage.filters import gaussian_filter1d, gaussian_filter
from scipy.stats import binned_statistic, entropy, norm
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

import sys
import os
import time
#import pickle
import dill as pickle

import concurrent.futures

from tqdm.notebook import tqdm
from collections import defaultdict
import itertools

import matplotlib.pyplot as plt
import seaborn as sns
#from IPython.display import display
sns.set(context='notebook', style='ticks', palette='bright', font='sans-serif', font_scale=1, color_codes=True, rc=None)
plt.rcParams['figure.figsize'] = (14, 8)
#print("SYS.PATH: ", sys.path[:3])
#sys.path.insert(0, r"C:\Users\User\[[Python]]\[AlexeyT]\PAC_PROJECT")

#from utility_functions import *
#from lfp_class import LFP
#from pac_class import MyPAC
#from patient_class import Patient

%run utility_functions.py
%run lfp_class.py
%run pac_class.py
%run patient_class.py


print("Succesfully imported libraries and modules\n")

Succesfully imported libraries and modules



In [2]:
with open("path_data.txt") as f:
    data_dir = f.readline()

p4_root_dir = os.path.join(data_dir, "Patient4")
#root_dir = r"H:\Alexey_Timchenko\PAC\Patient2"

In [4]:
p4 = Patient(name='Patient4', root_dir=p4_root_dir)
files = p4.find_bdf_files()
for filename in files:
    p4.scan_file_annotations(filename, update_file_conditions=True)

List of things to make sure before analysis: 
1) .bdf files are in patient folder (root_dir)
2) annotation files share the same name as .bdf files but with _annotations.txt suffix
3) annotations share the same naming principle: e.g. 1Day OFF RH (Com)
Looking for .bdf files in D:\Alex PAC\data\Patient4
Found ['Krasova_STN_postoperHP_0.5Hz_1 Day.bdf', 'Krasova_STN_postoperHP_0.5Hz_5 Day_Part 1.bdf', 'Krasova_STN_postoperHP_0.5Hz_5 Day_Part 2.bdf']
Reading  Krasova_STN_postoperHP_0.5Hz_1 Day_annotations.txt
       Onset  Duration           Annotation   Day L-DOPA          State
0     12.000      30.0    1Day OFF Rest (1)  1Day    OFF    [Rest, (1)]
1     55.000      80.0    1Day OFF Rest (2)  1Day    OFF    [Rest, (2)]
2    144.000     150.0    1Day OFF Rest (3)  1Day    OFF    [Rest, (3)]
3    314.000      60.0    1Day OFF RH (Com)  1Day    OFF    [RH, (Com)]
4    500.000      30.0  1Day OFF RH (NoCom)  1Day    OFF  [RH, (NoCom)]
5    588.000      40.0   1Day OFF RH (Pass)  1Day    OFF  

In [5]:
p4.get_preprocessed_lfps(verbose=False)

Started reading D:\Alex PAC\data\Patient4\Krasova_STN_postoperHP_0.5Hz_5 Day_Part 1.bdf
Channels: [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


Reading Channels: 100%|████████████████████████████████████████████████████████████████| 16/16 [01:06<00:00,  4.13s/it]


EEG A2-R_1-9[A2]
EEG F8-R_2C-15[F
EEG T4-R_2B-12[T
EEG T6-R_2A-10[T
EEG F4-R_3C-16[F
EEG C4-R_3B-14[C
EEG P4-R_3A-11[P
EEG O2-R_4-13[O2
EEG A1-R_1-1[A1]
EEG F7-R_2C-7[F7
EEG T3-R_2B-4[T3
EEG T5-R_2A-2[T5
EEG F3-R_3C-8[F3
EEG C3-R_3B-6[C3
EEG P3-R_3A-3[P3
EEG O1-R_4-5[O1]
Sampling frequency:  16000.0
Downsampling by the factor  8
New sampling frequency:  2000.0
Reading done, 95.7 sec
Started creating bipolar signals

R1-2C created
R1-2B created
R1-2A created
R2C-2B created
R2B-2A created
R2A-2C created
R2C-3C created
R2B-3B created
R2A-3A created
R3C-3B created
R3B-3A created
R3A-3C created
R4-3C created
R4-3B created
R4-3A created
L1-2C created
L1-2B created
L1-2A created
L2C-2B created
L2B-2A created
L2A-2C created
L2C-3C created
L2B-3B created
L2A-3A created
L3C-3B created
L3B-3A created
L3A-3C created
L4-3C created
L4-3B created
L4-3A created


Creating LFPs:   0%|          | 0/30 [00:00<?, ?it/s]

Started reading D:\Alex PAC\data\Patient4\Krasova_STN_postoperHP_0.5Hz_1 Day.bdf
Channels: [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


...

Creating LFPs:   0%|          | 0/30 [00:00<?, ?it/s]

Started reading D:\Alex PAC\data\Patient4\Krasova_STN_postoperHP_0.5Hz_5 Day_Part 2.bdf
Channels: [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


Reading Channels: 100%|████████████████████████████████████████████████████████████████| 16/16 [00:34<00:00,  2.19s/it]


EEG A2-R_1-9[A2]
EEG F8-R_2C-15[F
EEG T4-R_2B-12[T
EEG T6-R_2A-10[T
EEG F4-R_3C-16[F
EEG C4-R_3B-14[C
EEG P4-R_3A-11[P
EEG O2-R_4-13[O2
EEG A1-R_1-1[A1]
EEG F7-R_2C-7[F7
EEG T3-R_2B-4[T3
EEG T5-R_2A-2[T5
EEG F3-R_3C-8[F3
EEG C3-R_3B-6[C3
EEG P3-R_3A-3[P3
EEG O1-R_4-5[O1]
Sampling frequency:  16000.0
Downsampling by the factor  8
New sampling frequency:  2000.0
Reading done, 46.9 sec
Started creating bipolar signals

R1-2C created
R1-2B created
R1-2A created
R2C-2B created
R2B-2A created
R2A-2C created
R2C-3C created
R2B-3B created
R2A-3A created
R3C-3B created
R3B-3A created
R3A-3C created
R4-3C created
R4-3B created
R4-3A created
L1-2C created
L1-2B created
L1-2A created
L2C-2B created
L2B-2A created
L2A-2C created
L2C-3C created
L2B-3B created
L2A-3A created
L3C-3B created
L3B-3A created
L3A-3C created
L4-3C created
L4-3B created
L4-3A created


Creating LFPs:   0%|          | 0/30 [00:00<?, ?it/s]

In [6]:
p4.display_all_annotations()

----1Day----
------OFF------
     Onset  Duration          State
0     12.0      30.0    [Rest, (1)]
1     55.0      80.0    [Rest, (2)]
2    144.0     150.0    [Rest, (3)]
3    314.0      60.0    [RH, (Com)]
4    500.0      30.0  [RH, (NoCom)]
5    588.0      40.0   [RH, (Pass)]
6    668.0     120.0   [RH, (Hold)]
7    839.0      60.0    [LH, (Com)]
8    941.0      30.0  [LH, (NoCom)]
9   1000.0      30.0   [LH, (Pass)]
10  1073.5     120.0   [LH, (Hold)]
11  1235.5      30.0         [RHLH]
------ON------
       Onset  Duration          State
12  3834.500     180.0    [Rest, (1)]
13  4017.500      90.0    [Rest, (2)]
14  4147.000      80.0    [RH, (Com)]
15  4268.500      30.0  [RH, (NoCom)]
16  4333.501      30.0   [RH, (Pass)]
17  4395.500     120.0   [RH, (Hold)]
18  4589.500      80.0    [LH, (Com)]
19  4693.800      40.0  [LH, (NoCom)]
20  4760.500      30.0   [LH, (Pass)]
21  4824.500     120.0   [LH, (Hold)]
22  5007.000      30.0         [RHLH]
----5Day----
------OFF------
   

In [7]:
# For this patient 1Day without movement-related conditions (not enough data for 180sec)
new_condition_name = "1Day OFF Rest 180sec"
conditions_to_merge = ["1Day OFF Rest (2)", "1Day OFF Rest (3)"]
p4.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)

new_condition_name = "1Day ON Rest 180sec"
conditions_to_merge = ["1Day ON Rest (1)"]
p4.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


new_condition_name = "1Day OFF RH Move 180sec"
conditions_to_merge = ["1Day OFF " + s for s in ["RH (Com)", "RH (NoCom)", "RH (Hold)", "RH (Pass)"]]
p4.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


new_condition_name = "1Day OFF LH Move 180sec"
conditions_to_merge = ["1Day OFF " + s for s in ["LH (Com)", "LH (NoCom)", "LH (Hold)", "LH (Pass)"]]
p4.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


new_condition_name = "1Day ON RH Move 180sec"
conditions_to_merge = ["1Day ON " + s for s in ["RH (Com)", "RH (NoCom)", "RH (Hold)", "RH (Pass)"]]
p4.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)

new_condition_name = "1Day ON LH Move 180sec"
conditions_to_merge = ["1Day ON " + s for s in ["LH (Com)", "LH (NoCom)", "LH (Hold)", "LH (Pass)"]]
p4.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


## 5 DAY

new_condition_name = "5Day OFF Rest 180sec"
conditions_to_merge = ["5Day OFF Rest"]
p4.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)

new_condition_name = "5Day ON Rest 180sec"
conditions_to_merge = ["5Day ON Rest", "5Day ON RH (Pass)", "5Day ON LH (Pass)"]
p4.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)

p4.comment("Not enough data for 5Day ON Rest (140 sec). Added RH (Pass) and LH (Pass) to reach 180 sec target")


new_condition_name = "5Day OFF RH Move 180sec"
conditions_to_merge = ["5Day OFF " + s for s in ["RH (Com)", "RH (NoCom)", "RH (Hold)", "RH (Pass)"]]
p4.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


new_condition_name = "5Day OFF LH Move 180sec"
conditions_to_merge = ["5Day OFF " + s for s in ["LH (Com)", "LH (NoCom)", "LH (Hold)", "LH (Pass)"]]
p4.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


new_condition_name = "5Day ON RH Move 180sec"
conditions_to_merge = ["5Day ON " + s for s in ["RH (Com)", "RH (NoCom)", "RH (Hold)", "RH (Pass)"]]
p4.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)

new_condition_name = "5Day ON LH Move 180sec"
conditions_to_merge = ["5Day ON " + s for s in ["LH (Com)", "LH (NoCom)", "LH (Hold)", "LH (Pass)"]]
p4.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)

Adding LFP to Patient4 object. 
Condition: 1Day OFF Rest 180sec 
Placement: R2A-3A
Updating condition
Adding LFP to Patient4 object. 
Condition: 1Day ON Rest 180sec 
Placement: R2A-3A
Updating condition
Adding LFP to Patient4 object. 
Condition: 1Day OFF RH Move 180sec 
Placement: R2A-3A
Updating condition
Adding LFP to Patient4 object. 
Condition: 1Day OFF LH Move 180sec 
Placement: R2A-3A
Updating condition
Adding LFP to Patient4 object. 
Condition: 1Day ON RH Move 180sec 
Placement: R2A-3A
Updating condition
Adding LFP to Patient4 object. 
Condition: 1Day ON LH Move 180sec 
Placement: R2A-3A
Updating condition
Adding LFP to Patient4 object. 
Condition: 5Day OFF Rest 180sec 
Placement: R2A-3A
Updating condition
Adding LFP to Patient4 object. 
Condition: 5Day ON Rest 180sec 
Placement: R2A-3A
Updating condition
Adding LFP to Patient4 object. 
Condition: 5Day OFF RH Move 180sec 
Placement: R2A-3A
Updating condition
Adding LFP to Patient4 object. 
Condition: 5Day OFF LH Move 180sec 
Pla

In [8]:
for condition in p4.conditions:
    if "180sec" in condition:
        print(condition)

5Day ON RH Move 180sec
5Day ON Rest 180sec
1Day OFF Rest 180sec
1Day ON Rest 180sec
5Day OFF Rest 180sec
5Day OFF RH Move 180sec
5Day OFF LH Move 180sec
1Day ON LH Move 180sec
5Day ON LH Move 180sec
1Day OFF RH Move 180sec
1Day OFF LH Move 180sec
1Day ON RH Move 180sec


In [9]:
p4.save()

Saving Patient4 object to D:\Alex PAC\data\Patient4\Patient4.pkl ...
Pickling Patient4 without ['pac']
Done, 1.9653160572052002 sec
File size: 2330.36980342865 MB
Returning filepath for saved file


'D:\\Alex PAC\\data\\Patient4\\Patient4.pkl'